# Loop 3 Analysis: Comprehensive Ensemble from ALL Sources

Goal: Find the best configuration for each N from ALL available pre-optimized sources.

Current best: 70.676816 (gap to target: 1.75 points)
Target: 68.922808

In [1]:
import math
import numpy as np
import pandas as pd
from numba import njit
import os
from glob import glob

# Tree polygon template
@njit
def make_polygon_template():
    tw=0.15; th=0.2; bw=0.7; mw=0.4; ow=0.25
    tip=0.8; t1=0.5; t2=0.25; base=0.0; tbot=-th
    x = np.array([0,ow/2,ow/4,mw/2,mw/4,bw/2,tw/2,tw/2,-tw/2,-tw/2,-bw/2,-mw/4,-mw/2,-ow/4,-ow/2], np.float64)
    y = np.array([tip,t1,t1,t2,t2,base,base,tbot,tbot,base,base,t2,t2,t1,t1], np.float64)
    return x, y

@njit
def score_group(xs, ys, degs, tx, ty):
    n = xs.size
    V = tx.size
    mnx = 1e300; mny = 1e300; mxx = -1e300; mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c = math.cos(r); s = math.sin(r)
        xi = xs[i]; yi = ys[i]
        for j in range(V):
            X = c*tx[j] - s*ty[j] + xi
            Y = s*tx[j] + c*ty[j] + yi
            if X < mnx: mnx = X
            if X > mxx: mxx = X
            if Y < mny: mny = Y
            if Y > mxy: mxy = Y
    side = max(mxx - mnx, mxy - mny)
    return side * side / n

tx, ty = make_polygon_template()
_ = score_group(np.array([0.0]), np.array([0.0]), np.array([45.0]), tx, ty)
print('JIT compiled')

JIT compiled


In [2]:
def strip(a):
    return np.array([float(str(v).replace('s','')) for v in a], np.float64)

def load_submission(filepath):
    try:
        df = pd.read_csv(filepath)
        df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)
        configs = {}
        for n, g in df.groupby('N'):
            xs = strip(g['x'].to_numpy())
            ys = strip(g['y'].to_numpy())
            degs = strip(g['deg'].to_numpy())
            configs[n] = (xs, ys, degs)
        return configs
    except Exception as e:
        print(f'Error loading {filepath}: {e}')
        return None

print('Helper functions defined')

Helper functions defined


In [3]:
# Find ALL CSV files from snapshots
base_dir = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/'
all_csvs = glob(os.path.join(base_dir, '**/*.csv'), recursive=True)
print(f'Found {len(all_csvs)} CSV files in preoptimized directory')
for f in all_csvs:
    print(f'  {f}')

Found 30 CSV files in preoptimized directory
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/ensemble.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/submission.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/best_ensemble.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/telegram/72.49.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/telegram/71.97.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/telegram/telegram_extracted/72.49.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/telegram/telegram_extracted/71.97.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa25-public/submission_JKoT4.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa25-public/New_Tree_144_196.csv
  /home/nonroot/snapshots/santa-2025/2

In [4]:
# Load all sources and compute scores for each N
print('Loading all sources and computing per-N scores...')
print('='*80)

all_sources = {}
for csv_path in all_csvs:
    name = os.path.basename(csv_path)
    configs = load_submission(csv_path)
    if configs and len(configs) == 200:
        # Compute score for each N
        n_scores = {}
        for n in range(1, 201):
            if n in configs:
                xs, ys, degs = configs[n]
                n_scores[n] = score_group(xs, ys, degs, tx, ty)
        total = sum(n_scores.values())
        all_sources[csv_path] = {'configs': configs, 'n_scores': n_scores, 'total': total}
        print(f'{name}: {total:.6f}')
    else:
        print(f'{name}: SKIPPED (incomplete or error)')

print('='*80)
print(f'Loaded {len(all_sources)} complete sources')

Loading all sources and computing per-N scores...
ensemble.csv: 70.676102
submission.csv: 70.676501
santa-2025.csv: 70.676102


best_ensemble.csv: 70.676102
72.49.csv: 72.495739
71.97.csv: 71.972027
72.49.csv: 72.495739
71.97.csv: 71.972027


submission_JKoT4.csv: 72.489504
New_Tree_144_196.csv: 72.927920
submission_JKoT3.csv: 72.489488
santa2025_ver2_v61.csv: 72.951925
submission_JKoT2.csv: 72.489348


santa2025_ver2_v67.csv: 72.938567
santa2025_ver2_v76.csv: 72.826444
submission_70_936673758122.csv: 70.936674
santa2025_ver2_v65.csv: 72.935294


submission_70_926149550346.csv: 70.926150
santa2025_ver2_v66.csv: 72.938599
santa2025_ver2_v63.csv: 72.947427
santa2025_ver2_v69.csv: 72.850110
submission_JKoT1.csv: 72.489483


submission_opt1.csv: 70.990692
santa2025_ver2_v68.csv: 72.939233
santa-2025.csv: 70.676102
submission.csv: 70.676501


submission (77).csv: 72.135010
submission.csv: 72.935294
submission_sa.csv: 72.935294
submission_best.csv: 70.926150
Loaded 30 complete sources


In [5]:
# Find best config for each N across ALL sources
print('\nFinding best config for each N across all sources...')

best_for_n = {}  # n -> (score, source_path, config)
for source_path, data in all_sources.items():
    for n in range(1, 201):
        score = data['n_scores'].get(n, float('inf'))
        if n not in best_for_n or score < best_for_n[n][0]:
            best_for_n[n] = (score, source_path, data['configs'][n])

# Calculate ensemble score
ensemble_score = sum(best_for_n[n][0] for n in range(1, 201))
print(f'\nEnsemble score (best per N): {ensemble_score:.6f}')
print(f'Target: 68.922808')
print(f'Gap: {ensemble_score - 68.922808:.6f}')

# Show which sources contributed
source_counts = {}
for n in range(1, 201):
    src = os.path.basename(best_for_n[n][1])
    source_counts[src] = source_counts.get(src, 0) + 1

print('\nSource contributions:')
for src, count in sorted(source_counts.items(), key=lambda x: -x[1]):
    print(f'  {src}: {count} configs')


Finding best config for each N across all sources...

Ensemble score (best per N): 70.676102
Target: 68.922808
Gap: 1.753294

Source contributions:
  ensemble.csv: 200 configs


In [6]:
# Show top 20 N values with highest scores (most room for improvement)
print('\nTop 20 N values with highest scores (most room for improvement):')
n_scores_sorted = sorted([(n, best_for_n[n][0]) for n in range(1, 201)], key=lambda x: -x[1])
for n, score in n_scores_sorted[:20]:
    src = os.path.basename(best_for_n[n][1])
    print(f'  N={n:3d}: {score:.6f} (from {src})')


Top 20 N values with highest scores (most room for improvement):
  N=  1: 0.661250 (from ensemble.csv)
  N=  2: 0.450779 (from ensemble.csv)
  N=  3: 0.434745 (from ensemble.csv)
  N=  5: 0.416850 (from ensemble.csv)
  N=  4: 0.416545 (from ensemble.csv)
  N=  7: 0.399897 (from ensemble.csv)
  N=  6: 0.399610 (from ensemble.csv)
  N=  9: 0.387415 (from ensemble.csv)
  N=  8: 0.385407 (from ensemble.csv)
  N= 15: 0.379203 (from ensemble.csv)
  N= 10: 0.376630 (from ensemble.csv)
  N= 21: 0.376451 (from ensemble.csv)
  N= 20: 0.376057 (from ensemble.csv)
  N= 11: 0.375736 (from ensemble.csv)
  N= 22: 0.375258 (from ensemble.csv)
  N= 16: 0.374128 (from ensemble.csv)
  N= 26: 0.373997 (from ensemble.csv)
  N= 12: 0.372724 (from ensemble.csv)
  N= 13: 0.372323 (from ensemble.csv)
  N= 25: 0.372144 (from ensemble.csv)


In [7]:
# Check if ensemble has any improvements over current best
current_best_path = '/home/code/experiments/007_preoptimized_baseline/submission.csv'
current_configs = load_submission(current_best_path)

if current_configs:
    current_n_scores = {}
    for n in range(1, 201):
        xs, ys, degs = current_configs[n]
        current_n_scores[n] = score_group(xs, ys, degs, tx, ty)
    current_total = sum(current_n_scores.values())
    print(f'Current best total: {current_total:.6f}')
    
    # Find improvements
    improvements = []
    for n in range(1, 201):
        diff = current_n_scores[n] - best_for_n[n][0]
        if diff > 1e-9:
            improvements.append((n, diff, current_n_scores[n], best_for_n[n][0]))
    
    if improvements:
        print(f'\nFound {len(improvements)} improvements:')
        improvements.sort(key=lambda x: -x[1])
        for n, diff, old, new in improvements[:20]:
            src = os.path.basename(best_for_n[n][1])
            print(f'  N={n:3d}: {old:.6f} -> {new:.6f} (improvement: {diff:.6f}) from {src}')
        total_improvement = sum(x[1] for x in improvements)
        print(f'\nTotal potential improvement: {total_improvement:.6f}')
    else:
        print('No improvements found - current solution is already the best ensemble')

Current best total: 70.676816

Found 14 improvements:
  N=170: 0.342156 -> 0.341484 (improvement: 0.000672) from ensemble.csv
  N= 61: 0.358500 -> 0.358284 (improvement: 0.000217) from ensemble.csv
  N= 41: 0.368062 -> 0.367868 (improvement: 0.000193) from ensemble.csv
  N=171: 0.341592 -> 0.341449 (improvement: 0.000143) from ensemble.csv
  N= 77: 0.351174 -> 0.351114 (improvement: 0.000060) from ensemble.csv
  N=128: 0.343813 -> 0.343762 (improvement: 0.000051) from ensemble.csv
  N=114: 0.345639 -> 0.345621 (improvement: 0.000018) from ensemble.csv
  N= 43: 0.370042 -> 0.370040 (improvement: 0.000002) from ensemble.csv
  N=157: 0.341877 -> 0.341876 (improvement: 0.000001) from ensemble.csv
  N=137: 0.343246 -> 0.343245 (improvement: 0.000000) from ensemble.csv
  N=160: 0.339407 -> 0.339407 (improvement: 0.000000) from ensemble.csv
  N=103: 0.344118 -> 0.344118 (improvement: 0.000000) from ensemble.csv
  N=187: 0.340604 -> 0.340604 (improvement: 0.000000) from ensemble.csv
  N= 36: 0

In [8]:
# Check if the improved configs from ensemble.csv have overlaps
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.strtree import STRtree

getcontext().prec = 25
scale_factor = Decimal('1e15')

class ChristmasTree:
    def __init__(self, center_x, center_y, angle):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))

        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h

        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated,
                                          xoff=float(self.center_x * scale_factor),
                                          yoff=float(self.center_y * scale_factor))

def has_overlap(trees):
    if len(trees) <= 1:
        return False
    polygons = [t.polygon for t in trees]
    tree_index = STRtree(polygons)
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx != i and poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

# Check ensemble.csv configs for overlaps
ensemble_path = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/ensemble.csv'
ensemble_configs = load_submission(ensemble_path)

# Check the N values that could improve our score
improvement_ns = [170, 61, 41, 171, 77, 128, 114, 43, 157, 137, 160, 103, 187, 36]

print("Checking ensemble.csv configs for overlaps:")
valid_improvements = []
for n in improvement_ns:
    xs, ys, degs = ensemble_configs[n]
    trees = [ChristmasTree(xs[i], ys[i], degs[i]) for i in range(n)]
    has_ovlp = has_overlap(trees)
    status = "OVERLAP" if has_ovlp else "VALID"
    print(f"  N={n:3d}: {status}")
    if not has_ovlp:
        valid_improvements.append(n)

Checking ensemble.csv configs for overlaps:
  N=170: OVERLAP
  N= 61: OVERLAP
  N= 41: OVERLAP
  N=171: OVERLAP
  N= 77: OVERLAP
  N=128: VALID
  N=114: OVERLAP
  N= 43: OVERLAP
  N=157: OVERLAP
  N=137: OVERLAP
  N=160: VALID
  N=103: VALID
  N=187: VALID
  N= 36: VALID


In [9]:
# For the overlapping N values, find the best VALID config from all sources
print("Finding best VALID configs for overlapping N values...")

overlapping_ns = [170, 61, 41, 171, 77, 114, 43, 157, 137]

for n in overlapping_ns:
    print(f"\nN={n}:")
    best_valid_score = float('inf')
    best_valid_source = None
    
    for source_path, data in all_sources.items():
        xs, ys, degs = data['configs'][n]
        trees = [ChristmasTree(xs[i], ys[i], degs[i]) for i in range(n)]
        if not has_overlap(trees):
            score = data['n_scores'][n]
            if score < best_valid_score:
                best_valid_score = score
                best_valid_source = source_path
    
    if best_valid_source:
        print(f"  Best valid: {best_valid_score:.6f} from {os.path.basename(best_valid_source)}")
        print(f"  Current:    {current_n_scores[n]:.6f}")
        print(f"  Improvement: {current_n_scores[n] - best_valid_score:.6f}")
    else:
        print(f"  No valid config found in any source!")

Finding best VALID configs for overlapping N values...

N=170:


  Best valid: 0.341484 from submission.csv
  Current:    0.342156
  Improvement: 0.000672

N=61:


  Best valid: 0.358284 from submission.csv
  Current:    0.358500
  Improvement: 0.000217

N=41:
  Best valid: 0.367868 from submission.csv
  Current:    0.368062
  Improvement: 0.000193

N=171:


  Best valid: 0.341449 from submission.csv
  Current:    0.341592
  Improvement: 0.000143

N=77:


  Best valid: 0.351174 from submission_70_926149550346.csv
  Current:    0.351174
  Improvement: 0.000000

N=114:


  Best valid: 0.345621 from submission.csv
  Current:    0.345639
  Improvement: 0.000018

N=43:
  Best valid: 0.370040 from submission.csv
  Current:    0.370042
  Improvement: 0.000002

N=157:


  Best valid: 0.341876 from submission.csv
  Current:    0.341877
  Improvement: 0.000001

N=137:


  Best valid: 0.343246 from submission.csv
  Current:    0.343246
  Improvement: -0.000000


In [10]:
# Check scores of submissions from other snapshots
snapshot_submissions = [
    '/home/nonroot/snapshots/santa-2025/20992536951/code/submission.csv',
    '/home/nonroot/snapshots/santa-2025/21086827828/code/submission.csv',
    '/home/nonroot/snapshots/santa-2025/21108486172/code/submission.csv',
    '/home/nonroot/snapshots/santa-2025/21116303805/code/submission.csv',
    '/home/nonroot/snapshots/santa-2025/21121942239/code/submission.csv',
    '/home/nonroot/snapshots/santa-2025/21122900208/code/submission.csv',
    '/home/nonroot/snapshots/santa-2025/21122904233/code/submission.csv',
    '/home/nonroot/snapshots/santa-2025/21123768399/code/submission.csv',
    '/home/nonroot/snapshots/santa-2025/21129617858/code/submission.csv',
]

print("Scores of submissions from other snapshots:")
for path in snapshot_submissions:
    configs = load_submission(path)
    if configs and len(configs) == 200:
        total = sum(score_group(configs[n][0], configs[n][1], configs[n][2], tx, ty) for n in range(1, 201))
        print(f"  {path.split('/')[-3]}: {total:.6f}")
    else:
        print(f"  {path.split('/')[-3]}: INCOMPLETE")

Scores of submissions from other snapshots:
  20992536951: 118.183895
  21086827828: 116.975167
  21108486172: 70.734327


  21116303805: 70.676102
  21121942239: 70.676102
  21122900208: 70.676102
  21122904233: 118.230882


  21123768399: 70.676102
  21129617858: 70.676816


In [11]:
# Summary of findings
print("="*80)
print("SUMMARY OF ANALYSIS")
print("="*80)
print(f"\nCurrent best score: 70.676816")
print(f"Best available from sources: 70.676102")
print(f"Target: 68.922808")
print(f"Gap to target: {70.676102 - 68.922808:.6f}")
print(f"\nPotential improvement from better ensemble: {70.676816 - 70.676102:.6f}")
print("\nKey findings:")
print("1. The ensemble.csv has the best configs but many have overlaps")
print("2. submission.csv from snapshot 21116303805 has valid configs that are better")
print("3. The gap to target (1.75 points) is significant")
print("4. Need to run bbox3_advanced optimizer to try to improve further")
print("\nRecommended next steps:")
print("1. Create best valid ensemble from all sources")
print("2. Run bbox3_advanced optimizer on the ensemble")
print("3. Focus optimization on high-score N values (N=1,2,3,4,5)")

SUMMARY OF ANALYSIS

Current best score: 70.676816
Best available from sources: 70.676102
Target: 68.922808
Gap to target: 1.753294

Potential improvement from better ensemble: 0.000714

Key findings:
1. The ensemble.csv has the best configs but many have overlaps
2. submission.csv from snapshot 21116303805 has valid configs that are better
3. The gap to target (1.75 points) is significant
4. Need to run bbox3_advanced optimizer to try to improve further

Recommended next steps:
1. Create best valid ensemble from all sources
2. Run bbox3_advanced optimizer on the ensemble
3. Focus optimization on high-score N values (N=1,2,3,4,5)
